# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("\\Users\\twowh\Desktop\Module Challenges\Module 6 Challenge\python-api-challenge\Weatherpy\output_data\cities.csv")

# Display sample data
city_data_df.head()

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\twowh\AppData\Local\Temp\ipykernel_45196\2272874744.py:2: SyntaxWarning: invalid escape sequence '\D'
  city_data_df = pd.read_csv("\\Users\\twowh\Desktop\Module Challenges\Module 6 Challenge\python-api-challenge\Weatherpy\output_data\cities.csv")


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arar,30.9753,41.0381,1.01,29,0,2.57,SA,1734307618
1,1,puerto san jose,13.9256,-90.8244,29.03,74,0,0.00,GT,1734307619
2,2,avarua,-21.2078,-159.7750,27.03,83,39,4.63,CK,1734307620
3,3,port-aux-francais,-49.3500,70.2167,4.83,79,17,12.90,TF,1734307622
4,4,edd,13.9297,41.6925,25.96,59,2,7.05,ER,1734307623


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x ='Lng', 
    y = 'Lat', 
    geo=True, 
    tiles='OSM',  # You can choose different tile sources
    size='Humidity',  # Optional: size of the points based on population
    color='City',  # Optional: color points based on city
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_data_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)                               
]

# Drop any rows with null values
cleaned_narrowed_df = narrowed_city_data_df.dropna()

# Display sample data
cleaned_narrowed_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
225,225,waspan,14.7419,-83.9717,21.34,83,0,2.25,NI,1734307884
475,475,bobo-dioulasso,11.1771,-4.2979,21.25,26,0,3.09,BF,1734308187


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_narrowed_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
225,225,waspan,14.7419,-83.9717,21.34,83,0,2.25,NI,1734307884,None
475,475,bobo-dioulasso,11.1771,-4.2979,21.25,26,0,3.09,BF,1734308187,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel', #find hotels
    'apiKey': geoapify_key, #take api key from api_keys.py
    'limit': 1 #limit to the first hotel found
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row ['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    if name_address.status_code == 200:
        name_address = name_address.json()

        # Grab the first hotel from the results
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            print(f"No hotel found for coordinates ({latitude}, {longitude})")
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # Log API errors
        print(f"API Error: {name_address.status_code} - {name_address.text}")
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
waspan - nearest hotel: Hospedaje Vanessa
bobo-dioulasso - nearest hotel: Teria


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
225,225,waspan,14.7419,-83.9717,21.34,83,0,2.25,NI,1734307884,Hospedaje Vanessa
475,475,bobo-dioulasso,11.1771,-4.2979,21.25,26,0,3.09,BF,1734308187,Teria


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    x ='Lng', 
    y = 'Lat', 
    geo=True, 
    tiles='OSM',  # You can choose different tile sources
    size='Humidity',  # Optional: size of the points based on population
    color='City',  # Optional: color points based on city
    frame_width=700,
    frame_height=500,
    hover_cols = ['City', 'Hotel Name', 'Country', 'Lat', 'Lng', 'Humidity']
)


# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)